In [18]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from scipy import stats

In [19]:
categories = ['banana','beach','car','church','lighthouse','mountain','mug','plane']
path='../Analysis/human_rating_bycat'

In [20]:
for i, cate in enumerate(categories):
    col = [cate+'_'+str(i+1) for i in range(16)]
    new = pd.read_csv(path+'/'+cate+'.csv')
    new.columns = col
    if i == 0:
        old = new
    else:
        old = pd.concat([old,new],axis=1)
df = old

(Test) Use quantiles to remove outliers 

In [21]:
quantile_low = df.quantile(0.1)
quantile_high = df.quantile(0.9)
iqr = quantile_high - quantile_low

In [22]:
outliers = (df < quantile_low) | (df > quantile_high)
df_new = df[outliers.sum(axis=1) <= 12]

Compute 7 scorse and sort images by order in survey

In [23]:
alexnet_df=pd.read_csv('alexnet_scores_wordnet_id_closest_label.csv',index_col=0)
alexnet_df['typicality_score'] = np.round(alexnet_df['closest_prob'] * 100,2)
alexnet_df['7_score']= np.round(alexnet_df['typicality_score']*7/100,2)
alexnet_df['type'] = alexnet_df['type'].apply(lambda x: x.lower())

In [24]:
alexnet_df['number'] = alexnet_df['dir'].apply(lambda x: int(x.split('\\')[-1].split('.')[0]))
alexnet_df = alexnet_df.sort_values(['type','number']).reset_index(drop=True)

# Correlation with human ratings

Changes from original script: 
1. Use spearman rank correlation instead of pearson
2. Compute average human ratings first, and then compute correlation with neural net typicality scores

without dropping outliers

In [25]:
final_corr = dict()
for t in np.unique(alexnet_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    human_ratings = df[col_names] 
    net_ratings = alexnet_df[alexnet_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
#     net_scores = net_ratings['typicality_score'].values
    
    coef, p=stats.spearmanr(human_scores, net_scores)
#     coef, p=stats.pearsonr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.3004418262150221
beach:  0.2058823529411765
car:  0.24705882352941175
church:  0.351729307878893
lighthouse:  0.4223695036035242
mountain:  0.6975719327807334
mug:  0.06484894907460316
plane:  0.528329797190471


In [26]:
print(np.mean(list(final_corr.values())))

0.3522790616517294


after dropping outliers

In [27]:
final_corr = dict()
for t in np.unique(alexnet_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    
    human_ratings = df_new[col_names] 
    net_ratings = alexnet_df[alexnet_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
    coef, p=stats.spearmanr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.28718703976435933
beach:  0.17378516637919467
car:  0.21649508015035274
church:  0.3752760397871034
lighthouse:  0.41857048948152953
mountain:  0.6968356206287721
mug:  -0.042804532956051446
plane:  0.42773047454949503


In [28]:
print(np.mean(list(final_corr.values())))

0.3191344222230944
